# Seattle Airbnb Exploration
The data explored in this notebook was obtained from the [Seattle Airbnb Open Data](https://www.kaggle.com/airbnb/seattle/data) Kaggle site. 

In [23]:
import json
import os
from zipfile import ZipFile

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [21]:
# Download data - requires saved Kaggle API key to run: https://github.com/Kaggle/kaggle-api
!kaggle datasets download -d airbnb/seattle


  0%|          | 0.00/19.5M [00:00<?, ?B/s]
  5%|5         | 1.00M/19.5M [00:01<00:21, 918kB/s]
 10%|#         | 2.00M/19.5M [00:02<00:19, 921kB/s]
 15%|#5        | 3.00M/19.5M [00:03<00:18, 927kB/s]
 21%|##        | 4.00M/19.5M [00:04<00:17, 931kB/s]
 26%|##5       | 5.00M/19.5M [00:05<00:16, 934kB/s]
 31%|###       | 6.00M/19.5M [00:06<00:15, 936kB/s]
 36%|###5      | 7.00M/19.5M [00:07<00:13, 938kB/s]
 41%|####1     | 8.00M/19.5M [00:08<00:12, 939kB/s]
 46%|####6     | 9.00M/19.5M [00:10<00:12, 910kB/s]
 51%|#####1    | 10.0M/19.5M [00:11<00:10, 919kB/s]
 57%|#####6    | 11.0M/19.5M [00:12<00:09, 925kB/s]
 62%|######1   | 12.0M/19.5M [00:13<00:08, 930kB/s]
 67%|######6   | 13.0M/19.5M [00:14<00:07, 933kB/s]
 72%|#######1  | 14.0M/19.5M [00:15<00:06, 933kB/s]
 77%|#######7  | 15.0M/19.5M [00:16<00:05, 935kB/s]
 82%|########2 | 16.0M/19.5M [00:18<00:03, 937kB/s]
 87%|########7 | 17.0M/19.5M [00:19<00:02, 939kB/s]
 92%|#########2| 18.0M/19.5M [00:20<00:01, 910kB/s]
 98%|#########7| 19

In [24]:
# Unzip files
with ZipFile('seattle.zip', 'r') as zip_file:
    zip_file.extractall()